In [89]:
import pandas as pd
import numpy as np 


In [ ]:
# Primero hay que averigüar cuál es el label Id de los X-check de Italia. 
# ESto se logra leyendo el metadata. (cambiando 'raw' por 'metadata')
#Teniendo eso, ya se puede leer el contenido decodificándolo

In [145]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def getEmails():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):

        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # request a list of all the messages
    result = service.users().messages().list(userId='me', labelIds=['Label_1160941541567749419']).execute()

    # We can also pass maxResults to get any number of emails. Like this:
    # result = service.users().messages().list(maxResults=200, userId='me').execute()
    messages = result.get('messages')
    limit=17
    # messages is a list of dictionaries where each dictionary contains a message id.
    counter=0
    # iterate through all the messages
    for msg in messages:

        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id'], format='raw').execute()

        # Use try-except to avoid any Errors
        # try:
            # Get value of 'payload' from dictionary 'txt'
        # payload = txt['payload']
        # headers = payload['headers']
        # print(txt)
        # Look for Subject and Sender Email in the headers
#         for d in headers:
#             if d['name'] == 'Subject':
#                 subject = d['value']
#             if d['name'] == 'From':
#                 sender = d['value']

#         # The Body of the message is in Encrypted format. So, we have to decode it.
#         # Get the data and decode it with base 64 decoder.
#         if payload.get('parts')==None:
#             print(payload.get('parts'))
#         else:
#             parts = payload.get('parts')[0]
#             print(parts['body'])
#             # data = parts['body']['data']
#             data = data.replace("-","+").replace("_","/")
#             decoded_data = base64.b64decode(data)
#             print(d['value'])
#             # Now, the data obtained is in lxml. So, we will parse
#             # it with BeautifulSoup library
#             soup = BeautifulSoup(decoded_data , "lxml")
#             body = soup.body()

        # Printing the subject, sender's email and message
        # print("Subject: ", subject)
        # print("From: ", sender)
        # # print("Message: ", body)
        # print('\n')
        counter=counter+1
        if counter>=limit:
            return(txt)
            break
    # except:
    # 	counter=counter+1
        # print('failed_something')


text2 = getEmails()


In [148]:
def parse_msg(msg):
    if msg.get("payload").get("body").get("data"):
        return base64.urlsafe_b64decode(msg.get("payload").get("body").get("data").encode("ASCII")).decode("utf-8")
    return msg.get("snippet") 

In [149]:
msg_content=base64.urlsafe_b64decode(text2['raw'].encode('ASCII')).decode('utf-8')

In [150]:
text_file = open("data.txt", "w")
text_file.write(msg_content)
text_file.close()

In [152]:
text2.get('payload')

In [154]:
text2['snippet']

'SurveillanceMaster X-Check Report Process Time: 1/16/2023 8:06:07 PM Job Parameters SurveillanceMaster Version 5.23.2206.8 Report Date 1/16/2023 Time Window 16:00:00 to 18:59:59 Include Local Reels'